<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/llm_matrixInversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This simple example illustrates how you can integrate an LLM into a Python-based workflow to perform a mathematical task (matrix inversion) and then verify it with traditional computational methods. It also demonstrates how to prompt, parse, and validate LLM outputs, which is key to building robust AI applications.  Feel free to expand with additional logic, user interactions, or more sophisticated matrix problems.

Creating and Using Your OpenAI API Key:

To do this head to the OpenAI API Dashboard and create/login to an account. (https://platform.openai.com/assistants).
After logging into an account, on the left side of the dashboard select ‘API Keys’.
Next, click ‘Create new secret key’ in the top right corner of the page, from here you can create a name for your key, the project it is used for (if you have multiple keys and projects), and decide the permissions.
After deciding the settings for your key, click ‘Create secret key’ in the bottom right of the menu.
This will show your given key, make sure to copy your key and save it in a place of your choosing for later. This will be the only time you can access it. Afterwords, select ‘Done.’   

You have to add at least 5 dollars to your account. In most case you probably use only less than dollar in the whole semester.

In [58]:
!pip install openai requests -q


In [17]:
import openai
import ast  # for safely evaluating Pythonic list representations (optional)
import numpy as np  # to make matrix multiplication easier (optional)
import json
from openai import OpenAI

client = OpenAI(
    #replace your openai API key. make sure keep the key in a safe place,  colab and github do not save your jpynb file with the actual key.
    api_key="sk-proj-spiYK_**"
)

In [23]:



# Define a JSON schema for the function
functions = [
    {
        "name": "invert_matrix",
        "description": "Return the inverse of a 2x2 matrix in the 'inverted_matrix' field.",
        "parameters": {
            "type": "object",
            "properties": {
                "inverted_matrix": {
                    "type": "array",
                    "items": {
                        "type": "array",
                        "items": {"type": "number"}
                    },
                    "description": "A 2D array (list of lists) representing the inverted matrix"
                }
            },
            "required": ["inverted_matrix"]
        }
    }
]


In [24]:
def get_inverted_matrix_2x2(matrix_2x2):
    user_message = (
        f"Please invert the following 2x2 matrix: {matrix_2x2}. "
        "Return only a JSON object that calls the function `invert_matrix` "
        "with the key `inverted_matrix` as a 2D array."
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": user_message}
        ],
        functions=functions,
        function_call="auto"
        # ^ "auto" means the LLM can choose to call or not call the function
        #   "force" means we *force* it to call 'invert_matrix'
    )

    # The LLM will likely produce a special JSON payload referencing our function:
    # response.choices[0].message["function_call"] might look like:
    # {
    #   "name": "invert_matrix",
    #   "arguments": "{\"inverted_matrix\": [[0.6, -0.7], [-0.2, 0.4]]}"
    # }

    message = response.choices[0].message

    # Use attribute access instead of message.get(...)
    if message.function_call is not None:
        name = message.function_call.name
        arguments_str = message.function_call.arguments
        # parse the JSON string
        arguments = json.loads(arguments_str)
        return arguments["inverted_matrix"]  # e.g. [[0.6, -0.7], [-0.2, 0.4]]
    else:
        return None




In [25]:
def verify_inversion(original_matrix, candidate_inverse, tolerance=1e-7):
    """
    Verify if candidate_inverse is indeed the inverse of original_matrix.

    Args:
        original_matrix (list[list[float]]): 2x2 matrix
        candidate_inverse (list[list[float]]): 2x2 matrix (candidate inverse)
        tolerance (float): numerical tolerance for identity check.

    Returns:
        bool: True if product is approximately the identity matrix, False otherwise.
    """
    # Convert to numpy arrays
    A = np.array(original_matrix, dtype=float)
    A_inv_candidate = np.array(candidate_inverse, dtype=float)

    # Matrix multiply
    product = A.dot(A_inv_candidate)

    # Identity matrix
    I = np.eye(2)

    # Check if difference from the identity is within tolerance
    return np.allclose(product, I, atol=tolerance)


In [26]:
# Define a sample 2x2 matrix
matrix_2x2 = [[4, 7],
              [2, 6]]

# 1) LLM: Invert the matrix
inverted_candidate = get_inverted_matrix_2x2(matrix_2x2)
print("LLM's claimed inverse:", inverted_candidate)

# 2) Verify the candidate inverse
if inverted_candidate is not None:
    is_correct = verify_inversion(matrix_2x2, inverted_candidate)
    if is_correct:
        print("✅ The LLM's inverse is correct!")
    else:
        print("❌ The LLM's inverse is NOT correct.")


LLM's claimed inverse: [[0.6, -0.7], [-0.2, 0.4]]
✅ The LLM's inverse is correct!
